In [1]:
from pymongo import MongoClient
import pprint
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import copy
import re
from time import sleep
from selenium import webdriver
import numpy as np

In [2]:
carmax_url = 'https://www.carmax.com/cars/ford'
#carmax_url = 'https://www.carmax.com/'
#carmax_url = 'https://www.carvana.com/cars/ford'
#headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

In [3]:
client = MongoClient('localhost', 27017)
db = client['capstone1']
pages = db['Ford_TX']

In [4]:
Ford_files=['Ford_2013_2016.html','Ford_2017_2020.html']

In [5]:
url_links = []
for file in Ford_files:
    soup = BeautifulSoup(open("/home/ali/Documents/CapStone_Projects/CapStone1/" + file), "html.parser")
    for idx,link in enumerate(soup.find_all('article','car-tile')):
        url_links.append(link.a['href'])
        

In [7]:
car_mileage = []
car_price = []
car_mpg = []
car_model_year = []
car_model = []
car_make = []
car_color = []
car_transmission = []
car_engine_size = []
car_horse_power = []
car_engine_cylinder = []
car_msrp_price = []


i=0
for link in set(url_links):
    scrape_url = link
    print(link)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}
    r_1 = requests.get(scrape_url,headers=headers)
    soup = BeautifulSoup(r_1.text,'html.parser')
    car_mileage.append(soup.find_all('div','value')[1].text.strip())
    car_price.append(soup.find_all('div','value')[0].text.strip())
    
    car_mpg.append([np.nan if soup.find_all('div','mpg')==[] else soup.find('div','mpg').text])
    
    car_model_year.append(soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__year-make').text.strip(" ").split(" ",1)[0])
    car_model.append(soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__year-make').text.strip(" ").split(" ",1)[1])
    car_make.append(soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__model-trim').text.strip())
    a = soup.find_all('script')[6]
    a = str(a)
    car_color.append(a.split('ExteriorColor":"')[1].split('"')[0])
    car_transmission.append(a.split('Transmission":"')[1].split('"')[0])
    car_engine_size.append(float(a.split('EngineSize":"')[1].split('L')[0]))
    
    if a.split('Horsepower":')[1].split(',')[0] != 'null':
        var_HorsePower = float(a.split('Horsepower":')[1].split(',')[0])
    else:
        var_HorsePower = np.nan
    car_horse_power.append(var_HorsePower)
    
    car_engine_cylinder.append(int(a.split('EngineCylinders":')[1].split(',')[0]))
    car_msrp_price.append(float(a.split('Msrp":')[1].split(',')[0]))
    
    
#    pages.insert_one({'car_mileage': car_mileage, 
#                      'car_price': car_price, 
#                      'car_mpg': car_mpg ,
#                      'car_model_year': car_model_year,
#                      'car_model': car_model ,
#                      'car_make': car_make,
#                      'car_color': car_color,
#                      'car_transmission': car_transmission,
#                      'car_engine_size': car_engine_size,
#                      'car_horse_power': car_horse_power ,
#                      'car_engine_cylinder': car_engine_cylinder,
#                      'car_msrp_price':car_msrp_price})
    
    
    i+=1
    sleep(10)
    if i>3:
        break 

https://www.carmax.com/car/19859499


ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 603481682c1fbceb43410d3d, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused')>]>

In [ ]:
df = pd.DataFrame({'car_mileage': car_mileage, 
                   'car_price': car_price, 
                   'car_mpg': car_mpg ,
                   'car_model_year': car_model_year,
                   'car_model': car_model ,
                   'car_make': car_make,
                   'car_color': car_color,
                   'car_transmission': car_transmission,
                   'car_engine_size': car_engine_size,
                   'car_horse_power': car_horse_power ,
                   'car_engine_cylinder': car_engine_cylinder,
                   'car_msrp_price':car_msrp_price,
                                          })

In [8]:
df.to_csv("Ali.csv")

NameError: name 'df' is not defined